## Libraries

In [ ]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [ ]:
if IN_COLAB:
    !pip install transformers
    !pip install datasets
    !pip install evaluate
    !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 81.2 MB/s 
     |████████████████████████████████| 182 kB 92.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.7 MB/s 
     |████████████████████████████████| 132 kB 95.9 MB/s 
     |████████████████████████████████| 212 kB 58.6 MB/s 
     |████████████████████████████████| 127 kB 94.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 4.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

In [ ]:
import os
import torch

if IN_COLAB:
    root_path = '/content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA'
else:
    root_path = '/Users/kscaria/Library/CloudStorage/GoogleDrive-scariakevin1@gmail.com/My Drive/Knowledge/MSIT/Research/InstructABSA'
    
use_mps = True if torch.has_mps else False
os.chdir(root_path)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.utils import T5ATSC
from InstructABSA.data_prep import ModelReadyData

In [ ]:
rest_train_file_path = './Data/semeval14/ABSA_TrainData/Restaurants_Train_v2.csv'
laptops_train_file_path = './Data/semeval14/ABSA_TrainData/Laptop_Train_v2.csv'
rest_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Restaurants_Test_Gold.csv'
laptops_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Laptops_Test_Gold.csv'
rest_val_file_path = './Data/semeval14/ABSA_TrainData/Rest_val.csv'
laptops_val_file_path = './Data/semeval14/ABSA_TrainData/Lap_val.csv'

# Load the data
res_tr_df = pd.read_csv(rest_train_file_path)
lap_tr_df = pd.read_csv(laptops_train_file_path)
res_te_df = pd.read_csv(rest_test_file_path)
lap_te_df = pd.read_csv(laptops_test_file_path)
res_val_df = pd.read_csv(rest_val_file_path)
lap_val_df = pd.read_csv(laptops_val_file_path)

lap_tr_df =pd.concat([lap_tr_df, lap_val_df]).reset_index(drop = True)
res_tr_df =pd.concat([res_tr_df, res_val_df]).reset_index(drop = True)

In [ ]:
# Extract the aspect term and polarity for each review
model_data = ModelReadyData()
res_tr_df = model_data.extract_rowwise_aspect_polarity(res_tr_df, on='aspectTerms', by="term", min_val=1)
lap_tr_df = model_data.extract_rowwise_aspect_polarity(lap_tr_df, on='aspectTerms', by="term", min_val=1)
res_te_df = model_data.extract_rowwise_aspect_polarity(res_te_df, on='aspectTerms', by="term", min_val=1)
lap_te_df = model_data.extract_rowwise_aspect_polarity(lap_te_df, on='aspectTerms', by="term", min_val=1)

# Get the input text into the required format
bos_instruction_lap = """Definition: The output will be 'positive' if the aspect identified in the sentence contains a positive sentiment. If the sentiment of the identified aspect in the input is negative the answer will be 'negative'. 
Otherwise, the output should be 'neutral'. For aspects which are classified as noaspectterm, the sentiment is none.
Positive example 1-
input: I charge it at night and skip taking the cord with me because of the good battery life. The aspect is battery life.
output: positive
Positive example 2-
input: I even got my teenage son one, because of the features that it offers, like, iChat, Photobooth, garage band and more!. The aspect is garage band.
output: positive
Negative example 1-
input: Speaking of the browser, it too has problems. The aspect is browser.
output: negative
Negative example 2-
input: The keyboard is too slick. The aspect is keyboard.
output: negative
Neutral example 1-
input: I took it back for an Asus and same thing- blue screen which required me to remove the battery to reset. The aspect is battery.
output: neutral
Neutral example 2-
input: Nightly my computer defrags itself and runs a virus scan. The aspect is virus scan.
output: neutral
Now complete the following example-
input: """

bos_instruction_res = """Definition: The output will be 'positive' if the aspect identified in the sentence contains a positive sentiment. If the sentiment of the identified aspect in the input is negative the answer will be 'negative'. 
Otherwise, the output should be 'neutral'. For aspects which are classified as noaspectterm, the sentiment is none.
Positive example 1-
input: With the great variety on the menu , I eat here often and never get bored. The aspect is menu.
output: positive
Positive example 2- 
input: Great food, good size menu, great service and an unpretensious setting. The aspect is food.
output: positive
Negative example 1-
input: They did not have mayonnaise, forgot our toast, left out ingredients (ie cheese in an omelet), below hot temperatures and the bacon was so over cooked it crumbled on the plate when you touched it. The aspect is toast.
output: negative
Negative example 2-
input: The seats are uncomfortable if you are sitting against the wall on wooden benches. The aspect is seats.
output: negative
Neutral example 1-
input: I asked for seltzer with lime, no ice. The aspect is seltzer with lime.
output: neutral
Neutral example 2-
input: They wouldnt even let me finish my glass of wine before offering another. The aspect is glass of wine.
output: neutral
Now complete the following example-
input: """
delim_instruction = ' The aspect is '
eos_insrtuction = '.\noutput:'

res_tr_df = model_data.create_data_in_atsc_format(res_tr_df, 'raw_text', 'aspect', 'labels', bos_instruction_res, delim_instruction, eos_insrtuction)
lap_tr_df = model_data.create_data_in_atsc_format(lap_tr_df, 'raw_text', 'aspect', 'labels', bos_instruction_lap, delim_instruction, eos_insrtuction)
res_te_df = model_data.create_data_in_atsc_format(res_te_df, 'raw_text', 'aspect', 'labels', bos_instruction_res, delim_instruction, eos_insrtuction)
lap_te_df = model_data.create_data_in_atsc_format(lap_te_df, 'raw_text', 'aspect', 'labels', bos_instruction_lap, delim_instruction, eos_insrtuction)

In [ ]:
print(res_tr_df.iloc[0]['text'], res_tr_df.iloc[0]['labels'])

Definition: The output will be 'positive' if the aspect identified in the sentence contains a positive sentiment. If the sentiment of the identified aspect in the input is negative the answer will be 'negative'. 
Otherwise, the output should be 'neutral'. For aspects which are classified as noaspectterm, the sentiment is none.
Positive example 1-
input: With the great variety on the menu , I eat here often and never get bored. The aspect is menu.
output: positive
Positive example 2- 
input: Great food, good size menu, great service and an unpretensious setting. The aspect is food.
output: positive
Negative example 1-
input: They did not have mayonnaise, forgot our toast, left out ingredients (ie cheese in an omelet), below hot temperatures and the bacon was so over cooked it crumbled on the plate when you touched it. The aspect is toast.
output: negative
Negative example 2-
input: The seats are uncomfortable if you are sitting against the wall on wooden benches. The aspect is seats.
ou

In [ ]:
# Experimentation
experiment_id = 'laptops'
experiment_name = 'laptops_instruct_pos_neg_neut'
model_checkpoint = 'allenai/tk-instruct-base-def-pos'

# Create T5 utils object
t5_exp = T5ATSC(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATSC')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
id_dataset, ood_dataset, id_tokenized_dataset, ood_tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':8,
    'per_device_eval_batch_size':8,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'use_mps_device':use_mps
}

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-laptops_instruct_pos_neg_neut/checkpoints


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
# Train model
model_trainer = t5_exp.train(id_tokenized_dataset, **training_args)

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3964
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1984
  Number of trainable parameters = 247534848
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer device: cuda:0

Model training started ....


Epoch,Training Loss,Validation Loss
1,No log,0.211310
2,0.379400,0.217144
3,0.200200,0.260029
4,0.156000,0.356575


***** Running Evaluation *****
  Num examples = 1032
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1032
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1032
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1032
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-laptops_instruct_pos_neg_neut/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-laptops_instruct_pos_neg_neut/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-laptops_instruct_pos_neg_neut/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/all

In [ ]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

try:
    model_trainer
except:
    model_trainer = None

# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = id_tokenized_dataset, sample_set = 'train', trained_model_path = model_out_path)

# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = id_tokenized_dataset, sample_set = 'validation', trained_model_path = model_out_path)

# Get prediction labels - OOD Training set
ood_tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = ood_tokenized_dataset, sample_set = 'train', trained_model_path = model_out_path)

# Get prediction labels - OOD Testing set
ood_te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = ood_tokenized_dataset, sample_set = 'validation', trained_model_path = model_out_path)

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3964
  Batch size = 8


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-laptops_instruct_pos_neg_neut/checkpoints


***** Running Prediction *****
  Num examples = 1032
  Batch size = 8


The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 4809
  Batch size = 8


***** Running Prediction *****
  Num examples = 1328
  Batch size = 8


In [ ]:
# Add new column in the respective dataframes
id_tr_df = pd.DataFrame(id_dataset['train'])
id_te_df = pd.DataFrame(id_dataset['validation'])
ood_tr_df = pd.DataFrame(ood_dataset['train'])
ood_te_df = pd.DataFrame(ood_dataset['validation'])

id_tr_df['pred_labels'] = id_tr_pred_labels
id_te_df['pred_labels'] = id_te_pred_labels
ood_tr_df['pred_labels'] = ood_tr_pred_labels
ood_te_df['pred_labels'] = ood_te_pred_labels

# Metrics
print('In domain train accuracy: ', id_tr_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(id_tr_df))
print('In domain test accuracy: ', id_te_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(id_te_df))

ood_df = pd.concat([ood_tr_df, ood_te_df])
print('Out of domain accuracy: ', ood_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(ood_df))
print('Out of domain train accuracy: ', ood_tr_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(ood_tr_df))
print('Out of domain test accuracy: ', ood_te_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(ood_te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
id_tr_df.to_csv(os.path.join(dump_path, 'posnegneuttk_' + id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
id_te_df.to_csv(os.path.join(dump_path, 'posnegneuttk_' + id_te_filename), index = False)

ood_tr_filename = t5_exp.get_csv_filename(experiment_id)[2]
ood_tr_df.to_csv(os.path.join(dump_path, 'posnegneuttk_' + ood_tr_filename), index = False)

ood_te_filename = t5_exp.get_csv_filename(experiment_id)[3]
ood_te_df.to_csv(os.path.join(dump_path, 'posnegneuttk_' + ood_te_filename), index = False)

In domain train accuracy:  94.52573158425832
In domain test accuracy:  85.85271317829458
Out of domain accuracy:  80.31611536581391
Out of domain train accuracy:  79.55915990850488
Out of domain test accuracy:  83.05722891566265
